In [40]:
import yahoo_fin.stock_info as yf
import pandas as pd
from stage import fullPrint
import random
from datetime import datetime, timedelta
import numpy as np
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = nasdaqList + nyseList
tenYearSum = [0 for i in range(7)]
curSum = [0 for i in range(7)]
j = 0
random.shuffle(allList)
listOfStocks = [[] for i in range(4)]


In [41]:
from sklearn.model_selection import train_test_split
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)
# np.random.shuffle(train)
trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
trainSets = [trainSet1, trainSet2, trainSet3]

In [ ]:
# for i in range(len(allList)):
#     listOfStocks[i%4].append(allList[i])
#     # try:
#     #     df = pd.read_pickle("stockData/nyseNasdaq/"+allList[i]+".pkl")
#     # except:
#     #     pass
#     # try:
#     #     # drift = df.index[0].weekday()
#     #     # start = pd.to_datetime('2018-02-12',format = '%Y-%m-%d')
#     #     # end = pd.to_datetime('2022-01-10',format = '%Y-%m-%d')
#     #     # start = start + timedelta(days= drift)
#     #     # end = end + timedelta(days=drift)
#     #     a = df.iat[0,0]
#     #     b = df.iat[-1,0]
#     #     length = len(df.index)/52
#     #     # print(j%7,": first:",df.at[pd.to_datetime('2012-01-09',format = '%Y-%m-%d'),'close'],"second:",df.at[pd.to_datetime('2022-01-10',format = '%Y-%m-%d'),'close'])
#     #     if a != 0 and length > 3:
#     #         curSum[j%4] += (b/a)**(1/length)
#     #         print((b/a)**(1/length))
#     #         j = j + 1
#     #     # print(j%7,curSum[j%7], tenYearSum[j%7])
        
#     # except:
#     #     print(allList[i],"did not exist then")

In [78]:
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
        except:
            continue
        if not (df['close'] != 0).all():
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn
            
            
            

In [79]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = []
for l in trainSets:
    listOfDf.append(calculateGroupReturn(l))
print(listOfDf)


[             POAI      PHAS  IOBT     GOOGL      OBAS      SMIT      DLHC  \
1995:01  1.000000  1.000000     1  1.000000  1.000000  1.000000  1.000000   
1995:02  1.000000  1.000000     1  1.000000  1.000000  1.032051  1.000000   
1995:03  1.000000  1.000000     1  1.000000  1.000000  1.095238  1.000000   
1995:04  1.000000  1.000000     1  1.000000  1.000000  1.000000  1.000000   
1995:05  1.000000  1.000000     1  1.000000  1.000000  1.000000  1.000000   
...           ...       ...   ...       ...       ...       ...       ...   
2021:09  1.018692  1.000000     1  1.058944  1.042667  1.000000  1.122581   
2021:10  1.421569  1.030201     1  1.000000  1.000000  1.000000  1.018503   
2021:11  1.067797  1.059155     1  1.059094  1.000000  1.015345  1.229402   
2021:12  1.000000  1.000000     1  1.000000  1.084577  1.215139  1.000000   
2022:01  1.000000  1.105932     1  1.000000  1.014815  1.018553  1.363158   

             ALKT  IIIIU      AMOT  ...  BSCM       NWS     HTOOW  HTBX  E

In [ ]:
# sum = [0,0,0]
# index = 0
# countOfFirst = 0
# countOfStocks = 0
# for l in trainSets:
#     print(sum[index])
#     for ticker in l:
#         if ticker in growthOfStock:
#             sum[index] += growthOfStock[ticker][1]
#             if index == 0:
#                 countOfFirst += 1
#             countOfStocks += 1
#     print(sum[index])
#     sum[index] = sum[index] * (countOfFirst/countOfStocks)
#     print((countOfFirst/countOfStocks))
#     index += 1
#     countOfStocks = 0
# print(sum)
            

In [80]:
from statistics import mean
indexNum = 0
sum = [1,1,1]
for l in trainSets:
    listOfStockRet = []
    for index, element in listOfDf[indexNum].iterrows():
        #print(element.to_list())
        listOfStockRet = element.to_list()
        while 1.0 in listOfStockRet:
            listOfStockRet.remove(1.0)
        #print(listOfStockRet)
        if len(listOfStockRet) != 0:
            sum[indexNum] = sum[indexNum] * np.mean(listOfStockRet)
    indexNum += 1

In [81]:
print(sum)

[108837670329912.95, 64512260437766.02, 136316294167348.56]


In [51]:
for index, element in listOfDf[1].iterrows():
    print(element)

WLDN     1.0
ANIX     1.0
CCTSU    1.0
ENTFU    1.0
TCBI     1.0
        ... 
HCCI     1.0
WAVSU    1.0
AMST     1.0
EQIX     1.0
ITQRU    1.0
Name: 1995:01, Length: 1113, dtype: float64
WLDN     1.0
ANIX     1.0
CCTSU    1.0
ENTFU    1.0
TCBI     1.0
        ... 
HCCI     1.0
WAVSU    1.0
AMST     1.0
EQIX     1.0
ITQRU    1.0
Name: 1995:02, Length: 1113, dtype: float64
WLDN     1.0
ANIX     1.0
CCTSU    1.0
ENTFU    1.0
TCBI     1.0
        ... 
HCCI     1.0
WAVSU    1.0
AMST     1.0
EQIX     1.0
ITQRU    1.0
Name: 1995:03, Length: 1113, dtype: float64
WLDN     1.000000
ANIX     1.117647
CCTSU    1.000000
ENTFU    1.000000
TCBI     1.000000
           ...   
HCCI     1.000000
WAVSU    1.000000
AMST     1.000000
EQIX     1.000000
ITQRU    1.000000
Name: 1995:04, Length: 1113, dtype: float64
WLDN     1.0
ANIX     1.0
CCTSU    1.0
ENTFU    1.0
TCBI     1.0
        ... 
HCCI     1.0
WAVSU    1.0
AMST     1.0
EQIX     1.0
ITQRU    1.0
Name: 1995:05, Length: 1113, dtype: float64
WLDN     1